**Chapter 4 – Training Models**

- So far we have treated Machine Learning models and their training algorithms mostly like black boxes.
- Having a good understanding of how things work can help you pick the appropriate model, the right training algorithm, and provide you with a good set of hyperparameters for your task.
- Most of the topics discussed in this chapter will be essential in understanding, building, and training neural networks  

***Content***
1. Linear Regression
2. Polynomial Regression.
3. Regularized Linear Models
4. Logistic Regression.


# 0. Setup Python

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.7 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥1.01.

In [5]:
# Python ≥3.11 is required
import sys
assert sys.version_info >= (3, 11)

# Scikit-Learn ≥1.01 is required
from packaging import version
import sklearn
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")


# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)


# 1. Linear Regression

In Chapter 1 we looked at a simple regression model of life satisfaction:

$$
LifeSatisfaction = θ_0 + θ_1 × GDPPerCapita
$$

More generally: 

$$
\hat{y} = θ_0 + θ_1x_1 + θ_2x_2 +⋯+ θ_nx_n
$$

In this equation: 

- $\hat{y}$ is the predicted value.
- $n$ is the number of features.
- $x_i$ is the i<sup>th</sup> feature value.
- $\theta_j$ is the j<sup>th</sup> model parameter (including the bias term θ<sub>0</sub> and the feature weights θ<sub>1</sub>, θ<sub>2</sub>, ⋯, θ<sub>n</sub>).

This can be written much more concisely using a vectorized form as:

$$
\hat{y} = h_θ(\mathbf{x})  = \mathbf{θ} \cdot \mathbf{x}
$$

In this equation:
- $\mathbf{θ}$ is the model’s parameter vector, containing the bias term _θ<sub>0</sub>_ and the feature weights _θ<sub>1</sub>_ to _θ<sub>n</sub>_.
- $\mathbf{x}$ is the instance’s feature vector, containing x<sub>0</sub> to x<sub>n</sub> , with x<sub>0</sub> always equal to 1.
- $\mathbf{θ} \cdot \mathbf{x}$ is the dot product of the vectors $\mathbf{θ}$ and $\mathbf{x}$, which is equal to $θ_0x_0 + θ_1x_1 + θ_2x_2 +⋯+ θ_nx_n$.
- $h$ is the hypothesis function, using the model parameters $\mathbf{θ}$.

In Machine Learning, vectors are often represented as column vectors, which are 2D
arrays with a single column. If $\mathbf{θ}$ and $\mathbf{x}$ are column vectors, then the prediction is
$\hat{y} = \mathbf{θ}^T\mathbf{x}$, where $\mathbf{θ}^T$ is the transpose of $\mathbf{θ}$ (a row vector instead of a column vector) and
$\mathbf{θ}^T\mathbf{x}$ is the matrix multiplication of $\mathbf{θ}^T$ and $\mathbf{x}$. It is of course the same prediction, except
that it is now represented as a single-cell matrix rather than a scalar value. In this course we
will use this notation to avoid switching between dot products and matrix
multiplications.  

To train a Linear Regression model, we need to find the value of $\mathbf{θ}$ that minimizes the RMSE. In practice, it is simpler to minimize the mean squared error (MSE) than the RMSE, and it leads to the same result (because the value that minimizes a function also minimizes its square root). 

The MSE of a Linear Regression hypothesis $h_θ$ on a training set $\mathbf{X}$ is calculated using the equation 

$$
MSE(\mathbf{X},h_θ) = \frac{1}{m}\sum_{i=1}^{m}(\mathbf{θ}^T \mathbf{x}^{(i)} - {y}^{(i)} )^2
$$

with $m$ the number of instances in the training set. 






Let's plot some linear-looking data to test the algorithms on. 

In [6]:
import numpy as np

X = 2 * np.random.rand(100, 1) # 100 uniform random numbers in [0, 2]
y = 4 + 3 * X + np.random.randn(100, 1)  # Gaussian noise: N(0, 1): mean = 0, std = 1

In [7]:
X[:5], y[:5]

In [8]:
plt.figure(figsize=(6, 4))
plt.plot(X, y, "b.")
plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.axis([0, 2, 0, 15])
plt.grid()
plt.show()

Performing Linear Regression using Scikit-Learn is simple:

In [9]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_

This is very close to 3*x + 4.

It's easy to visualize the fitted line: just predict two x values

In [10]:
X_new = np.array([[0], [2]])
y_predict = lin_reg.predict(X_new)

In [11]:
y_predict

In [12]:
plt.figure(figsize=(6, 4))  # extra code – not needed, just formatting
plt.plot(X_new, y_predict, "r-", label="Predictions")
plt.plot(X, y, "b.")
plt.axis([0, 2, 0, 15])
# extra code – beautifies and saves Figure 4–2
plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.grid()
plt.legend(loc="upper left")

plt.show()

The `LinearRegression` class is based on the `scipy.linalg.lstsq()` function (the name stands for "least squares", referring to MSE).  
Please note that you could also use the SVD decomposition or the pseudoinverse (see course: Mathematics for Machine Learning)  
Calculate $\mathbf{X}^+\mathbf{y}$, where $\mathbf{X}^{+}$ is the _pseudoinverse_ of $\mathbf{X}$ (specifically the Moore-Penrose inverse).  

To be able to do so, you need to extend the X with a dummy variable X_0

In [13]:
from sklearn.preprocessing import add_dummy_feature
X_b = add_dummy_feature(X)  # add x0 = 1 to each instance

In [14]:
theta_best_svd, residuals, rank, s = np.linalg.lstsq(X_b, y, rcond=1e-6)
theta_best_svd

In [15]:
np.linalg.pinv(X_b) @ y

## Computational Complexity

The approach used by Scikit-Learn’s LinearRegression class is about $O(n^2)$. **If you double the number of features, you multiply the computation time by roughly 4**. It gets very slow when the number of features grows large (e.g., 100,000). On the positive side, **it is linear with regard to the number of instances** in the training set ($O(m)$), so it handles large training sets efficiently, provided they can fit in memory.

Also, **once you have trained your Linear Regression model, predictions are very fast: the computational complexity is linear with regard to both the number of
instances you want to make predictions on and the number of features**. In other words, making predictions on twice as many instances (or twice as
many features) will take roughly twice as much time. 

Now we will look at a very different way to train a Linear Regression model, which is better suited for cases where there are a large number of features or
too many training instances to fit in memory.

# Gradient Descent
_Gradient Descent_ is a generic optimization algorithm capable of finding optimal solutions to a wide range of problems. The general idea of Gradient
Descent is to tweak parameters iteratively in order to minimize a cost function.

Concretely, you start by filling $\mathbf{θ}$ with random values (this is called random initialization). Then you improve it gradually, taking one baby step at a time, each step attempting to decrease the cost function (e.g., the MSE), until the algorithm converges to a minimum. The learning step size is proportional to
the slope of the cost function, so the steps gradually get smaller as the parameters approach the
minimum

![](img/gradient_descent.png)

An important parameter in Gradient Descent is the size of the steps, determined by the _learning rate_ hyperparameter:  
- If the learning rate is too small, then the algorithm will have to go through many iterations to converge, which will take a long time.
- If the learning rate is too high, you might jump across the valley and end up on the other side, possibly even higher up than you were before.


## Batch Gradient Descent

To implement Gradient Descent, you need to compute the gradient of the cost function with regard to each model parameter $θ_j$. 
- In other words, you need to calculate how much the cost function will change if you change $θ_j$ just a little bit.
- This is called a _partial derivative_. It is like asking “What is the slope of the mountain under my feet if I face east?” and then asking the
same question facing north (and so on for all other dimensions). 

Notice that this approach involves calculations over the full training set X, at each Gradient Descent step! This is why the algorithm is called Batch Gradient Descent: 
- It uses the whole batch of training data at every step. 
- As a result it is terribly slow on very large training sets.

Let’s look at a quick implementation of this algorithm:


In [16]:
eta = 0.1  # learning rate
n_epochs = 1000
m = len(X_b)  # number of instances

np.random.seed(42)
theta = np.random.randn(2, 1)  # randomly initialized model parameters

for epoch in range(n_epochs):
    gradients = 2 / m * X_b.T @ (X_b @ theta - y)  # vectorized form of the gradient equation (4-5) in the book (@ is matrix multiplication)
    theta = theta - eta * gradients

The trained model parameters:

In [17]:
theta

Please note that this is the exact same solution as the linear regression found. 
Look at the next figure to see what the effect is of a different learning rate. The figure shows the first 20 steps of Gradient Descent using three different learning rates.  
The line at the bottom of each plot represents the random starting point, then each epoch is represented by a darker and darker line.
![](img/gradient_descent_plot.png)

## Stochastic Gradient Descent

To overcome the problem of training the whole batch at every step, _Stochastic Gradient Descent_ picks a random instance in the training set at every step
and computes the gradients based only on that single instance.

- This makes the algorithm much faster because it has very little data to manipulate at every iteration.
- It also makes it possible to train on huge training sets, since only one instance needs to be in memory at each iteration. 
- On the other hand, due to its stochastic (i.e., random) nature, this algorithm is much less regular than Batch Gradient Descent: instead of gently
decreasing until it reaches the minimum, the cost function will bounce up and down, decreasing only on average:

![](img/sgd.png)

In practice, with stochastic gradient descent we go through a number of rounds (called _epochs_). 
- In each epoch we iterate through $m$ (= number of instances in the training set) randomly selected instances. 
- At each such an iteration we calculate the gradient and try to approach the minimum a bit closer. 
- So we have a nested loop:
```
  for epoch in range(n_epochs):  
    for iteration in range(m):
    # calculate gradient for a randomly (stochastic) chosen instance
```

**The number of epochs is an important hyperparameter for algorithms that use stochastic gradient descent.**

To perform Linear Regression using Stochastic GD with Scikit-Learn, you can use the `SGDRegressor` class, which defaults to optimizing the squared
error cost function. The following code runs for maximum 1.000 epochs or until the loss drops by less than 0.00001 during one epoch (`max_iter=1000,
tol=1e-5`). It starts with a learning rate of 0.01 (`eta0=0.01`). Lastly, it does not use
any regularization (penalty=None; more details on this shortly):  

In [18]:
y[:5]

In [19]:
y.ravel()[:5] # ravel() returns a flattened array

In [20]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-5, penalty=None, eta0=0.01,n_iter_no_change=100, random_state=42)
# n_iter_no_change=100 means that the training will stop after 100 epochs without improvement
sgd_reg.fit(X, y.ravel())  # y.ravel() because fit() expects 1D targets (not 2D)

In [21]:
sgd_reg.intercept_, sgd_reg.coef_

This is very close to the result of the linear regression above.  
The resulting image below shows the first 20 steps of training, illustrating the irregularity in the steps in the process.
<img src="img/sgd_plot.png" width="50%">

## Mini-batch Gradient Descent
The last Gradient Descent algorithm we will look at is called Mini-batch Gradient
Descent. It is the compromise between Batch and Stochastic Gradient Descent:
at each step, instead of computing the gradients based on the full training set (as
in Batch GD) or based on just one instance (as in Stochastic GD), Mini-batch GD
computes the gradients on small random sets of instances called mini-batches. The
main advantage of Mini-batch GD over Stochastic GD is that you can get a performance
boost from hardware optimization of matrix operations, especially when using
Graphical Processing Units (GPUs).
<img src="img/gradient_descent_paths_plot.png" width="50%"/>

# 2. Polynomial Regression

What if your data is more complex than a straight line? 
- You can use a linear model to fit nonlinear data
- Add powers of each feature as new features
- Train a linear model on this extended set of features.  
This technique is called _Polynomial Regression_.

In [22]:
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)

In [23]:
plt.plot(X, y, "b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([-3, 3, 0, 10])
plt.show()

In [24]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
# include_bias=False means that the PolynomialFeatures class will not add the bias term (x0 = 1) for us
X_poly = poly_features.fit_transform(X)
X_poly[0]

`X_poly` now contains the original feature of X plus the square of this feature. Now you can fit a `LinearRegression` model to this extended training data:

In [25]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
lin_reg.intercept_, lin_reg.coef_

In [26]:
X_new=np.linspace(-3, 3, 100).reshape(100, 1)
X_new_poly = poly_features.transform(X_new)
y_new = lin_reg.predict(X_new_poly)
plt.plot(X, y, "b.")
plt.plot(X_new, y_new, "r-", linewidth=2, label="Predictions")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.legend(loc="upper left", fontsize=14)
plt.axis([-3, 3, 0, 10])
plt.show()

Not bad: the model estimates $\hat{y} = 0.56x_1^2 + 0.93x_1 + 1.78$ when in fact the original function was $y = 0.5x_1^2 + 1.0x_1 + 2.0$ + Gaussian noise. 

Note that when there are multiple features (e.g. $x_1$ and $x_2$), Polynomial Regression is capable of finding relationships between features (which is something a plain Linear Regression model cannot do). This is made possible by the fact that `PolynomialFeatures` also adds all combinations of features up to the given
degree. For example, if there were two features $a$ and $b$, `PolynomialFeatures` with degree=3 would not only add the features $a^2$, $a^3$, $b^2$ and $b^3$, but also the combinations $ab$, $a^2 b$, and $ab^2$.

High-degree Polynomial Regression can fit the training data much better than plain Linear Regression.

- The high-degree Polynomial Regression model is severely overfitting the training data.
- The linear model is underfitting it.

# Learning Curves

If you perform high-degree Polynomial Regression, you will likely fit the training data much better than with plain Linear Regression.  
Take a look at the figure below where we fit a 300-degree polynomial, a quadratic model (second degree) and a pure linear model (first degree) through the training set.  
Notice how the 300-degree polynomial model wiggles around to get as close as possible to the training instances. 

In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

plt.figure(figsize=(6, 4))

for style, width, degree in (("r-+", 2, 1), ("b--", 2, 2), ("g-", 1, 300)):
    polybig_features = PolynomialFeatures(degree=degree, include_bias=False)
    std_scaler = StandardScaler()
    lin_reg = LinearRegression()
    polynomial_regression = make_pipeline(polybig_features, std_scaler, lin_reg)
    polynomial_regression.fit(X, y)
    y_newbig = polynomial_regression.predict(X_new)
    label = f"{degree} degree{'s' if degree > 1 else ''}"
    plt.plot(X_new, y_newbig, style, label=label, linewidth=width)

plt.plot(X, y, "b.", linewidth=3)
plt.legend(loc="upper left")
plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.axis([-3, 3, 0, 10])
plt.grid()
plt.show()

This high-degree Polynomial Regression model is severely overfitting the training data, while the linear model is underfitting it.  
- Quadratic model generalizes best (makes sense: data generated using a quadratic model)  
- Normally, this information is not there, so how complex should the model be?  
- How to know when model is overfitting or underfitting?  
=> Chapter 2: cross-validation!!!  
- Overfitting: model performs well on training data but generalizes poorly on validation data 
- Underfitting: Poor performance on both 

### Learning curves  
Learning curves are plots of the model’s training error and validation error as a function of the training iteration.  
Evaluate the model at regular intervals during training on both the training set and the validation set, and plot the results.  
Scikit-Learn has a useful `learning_curve()` function to help with this: it trains and evaluates the model using cross-validation.  
By default it retrains the model on growing subsets of the training set.  
The function returns the training set sizes at which it evaluated the model, and the training and validation scores it
measured for each size and for each cross-validation fold.  
See the 2 figures below, which one is underfitting/overfitting? Why?  
If there is a gap between the curves, it means that the model performs significantly better on the training data than on the validation data, which is the hallmark of an overfitting model.

In [28]:
from sklearn.model_selection import learning_curve

print(len(X), len(y))

train_sizes, train_scores, valid_scores = learning_curve(
    LinearRegression(), X, y, train_sizes=np.linspace(0.01, 1.0, 40), cv=5,
    scoring="neg_root_mean_squared_error")
train_errors = -train_scores.mean(axis=1)
valid_errors = -valid_scores.mean(axis=1)
# learning_curve() returns the scores of the model on the training set and the validation set
# the parameter scoring="neg_root_mean_squared_error" means that the function will return the negative RMSE
# so we need to negate the scores to get the RMSE
# np.linspace(0.01, 1.0, 40) generates 40 evenly spaced training set sizes between 1% and 100% of the full training set size
# the full training set size = 4/5 of the total number of instances (80 instances) because of cv=5, which means that we are using 5-fold cross-validation

print(train_sizes)

plt.figure(figsize=(6, 4))  # extra code – not needed, just formatting
plt.plot(train_sizes, train_errors, "r-+", linewidth=2, label="train")
plt.plot(train_sizes, valid_errors, "b-", linewidth=3, label="valid")

# extra code – beautifies and saves Figure 4–15
plt.xlabel("Training set size")
plt.ylabel("RMSE")
plt.grid()
plt.legend(loc="upper right")
plt.axis([0,80, 0, 2.5])
plt.show()

This model is **underfitting**. To see why, first let’s look at the training error. When there
are just one or two instances in the training set, the model can fit them perfectly,
which is why the curve starts at zero. But as new instances are added to the training
set, it becomes impossible for the model to fit the training data perfectly, both because
the data is noisy and because it is not linear at all. So the error on the training
data goes up until it reaches a plateau, at which point adding new instances to the
training set doesn’t make the average error much better or worse. Now let’s look at
the validation error. When the model is trained on very few training instances, it is
incapable of generalizing properly, which is why the validation error is initially quite
large. Then, as the model is shown more training examples, it learns, and thus the
validation error slowly goes down. However, once again a straight line cannot do a
good job of modeling the data, so the error ends up at a plateau, very close to the
other curve.

If your model is underfitting the training data, adding more training
examples will not help. You need to use a better model or come
up with better features.

Now let's look at the learning curves for a 10 dimensional polynomial.

In [29]:
from sklearn.pipeline import make_pipeline

polynomial_regression = make_pipeline(
    PolynomialFeatures(degree=10, include_bias=False),
    LinearRegression())

train_sizes, train_scores, valid_scores = learning_curve(
    polynomial_regression, X, y, train_sizes=np.linspace(0.01, 1.0, 40), cv=5,
    scoring="neg_root_mean_squared_error")
# train_sizes=np.linspace(0.01, 1.0, 40) means that the training set size will be 1%, 2%, 3%, ..., 100% of the training set

In [30]:
train_errors = -train_scores.mean(axis=1)
valid_errors = -valid_scores.mean(axis=1)

plt.figure(figsize=(6, 4))
plt.plot(train_sizes, train_errors, "r-+", linewidth=2, label="train")
plt.plot(train_sizes, valid_errors, "b-", linewidth=3, label="valid")
plt.legend(loc="upper right")
plt.xlabel("Training set size")
plt.ylabel("RMSE")
plt.grid()
plt.axis([0, 80, 0, 2.5])

plt.show()

These learning curves look a bit like the previous ones, but there are two very
important differences:
- The error on the training data is much lower than before.
- There is a gap between the curves. This means that the model performs significantly
better on the training data than on the validation data, which is the hallmark of an overfitting model. If you used a much larger training set, however,
the two curves would continue to get closer.

### The Bias/Variance Trade-off
Statistics/Machine Learning: a model’s generalization error is the sum of three very different errors:  
 
**Bias:** Wrong assumptions, e.g. linear while it is actually quadratic. A high-bias model is most likely to underfit the training data.

**Variance:** 
Model’s excessive sensitivity to small variations in the training data.  
A model with many degrees of freedom (e.g. high-degree polynomial) is likely to have high variance and thus overfit the training data.

**Irreducible error:**  
Noisiness of the data itself. Reduction only by cleaning up the data (e.g., fix the data sources, such as broken sensors, or detect and remove outliers).  

Increasing a model’s complexity will typically increase its variance and reduce its bias.
Conversely, reducing a model’s complexity increases its bias and reduces its variance => **trade-off**.

# 3. Regularized Linear Models

- A good way to reduce overfitting is to regularize the model (i.e., to constrain it): the fewer degrees of freedom it has, the harder it will be for it to overfit the data. 
- A simple way to regularize a polynomial model is to reduce the number of polynomial degrees.
- For a linear model, regularization is typically achieved by constraining the weights of the model.

## Ridge Regression

Ridge Regression is a regularized version of Linear Regression: a regularization term is added to the cost function.

$$
J(\theta) = MSE(\theta) + \alpha\frac{1}{2}\sum_{i=1}^{n}\theta_i^2
$$

The hyperparameter *α* controls how much you want to regularize the model.
- If *α = 0*, then Ridge Regression is just Linear Regression. 
- If *α* is very large, then all weights end up very close to zero and the result is a flat line going through the data’s mean (because $θ_1$ to $θ_n$ will tend to zero).

Note that the bias term $θ_0$ is not regularized (the sum starts at *i = 1*, not *0*).

**WARNING**  
It is important to scale the data (e.g., using a StandardScaler) before performing
Ridge Regression, as it is sensitive to the scale of the input features. This is true for most regularized models.

The figures below show several Ridge models trained on some linear data using different $α$ values.

In [32]:
np.random.seed(42)
m = 20
X = 3 * np.random.rand(m, 1)
y = 1 + 0.5 * X + np.random.randn(m, 1) / 1.5
X_new = np.linspace(0, 3, 100).reshape(100, 1) # 100 evenly spaced numbers in [0, 3]

In [33]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1, solver="cholesky", random_state=42) # for now don't worry about the solver parameter
ridge_reg.fit(X, y)
ridge_reg.predict([[1.5]])

In [34]:
ridge_reg = Ridge(alpha=1, solver="sag", random_state=42)
ridge_reg.fit(X, y)
ridge_reg.predict([[1.5]])

Linear (left) and a 10th degree polynomial (right) models, both with various levels of
ridge regularization: 

In [35]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

def plot_model(model_class, polynomial, alphas, **model_kargs):
    for alpha, style in zip(alphas, ("b-", "g--", "r:")):
        model = model_class(alpha, **model_kargs) if alpha > 0 else LinearRegression()
        if polynomial:
            model = Pipeline([
                    ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
                    ("std_scaler", StandardScaler()),
                    ("regul_reg", model),
                ])
        model.fit(X, y)
        y_new_regul = model.predict(X_new)
        lw = 2 if alpha > 0 else 1
        plt.plot(X_new, y_new_regul, style, linewidth=lw, label=r"$\alpha = {}$".format(alpha))
    plt.plot(X, y, "b.", linewidth=3)
    plt.legend(loc="upper left", fontsize=15)
    plt.xlabel("$x_1$", fontsize=18)
    plt.axis([0, 3, 0, 3.5])
    plt.grid()

plt.figure(figsize=(9, 3.5))
plt.subplot(121)
plot_model(Ridge, polynomial=False, alphas=(0, 10, 100), random_state=42)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.subplot(122)
plot_model(Ridge, polynomial=True, alphas=(0, 10**-5, 1), random_state=42)

plt.show()

Note how increasing $α$ leads to flatter (i.e., less extreme, more reasonable) predictions, thus reducing overfitting. 

## Lasso Regression

Least Absolute Shrinkage and Selection Operator Regression **(Lasso Regression)** is another regularized version of Linear Regression.  
Just like Ridge Regression, it adds a regularization term to the cost function, but it uses the ℓ1 norm of the weight vector instead of the square of the ℓ2 norm.  
Lasso Regression cost function:
$$
J(\theta) = MSE(\theta) + 2\alpha \sum_{i=1}^{n} |\theta_i|
$$

An important characteristic of Lasso Regression is that it **tends to eliminate the weights of the least important features (i.e., set them to zero)**.  
For example, the dashed line in the righthand plot below (with α = 0.01) looks roughly cubic: all the weights for the high-degree polynomial features are equal to zero.  
In other words, Lasso Regression automatically performs feature selection and outputs a sparse model (i.e., with few nonzero feature weights). T

The figure below shows the same thing as before but replaces Ridge models with Lasso models and uses different α values.

In [36]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)
lasso_reg.predict([[1.5]])

In [37]:
# extra code – this cell generates and saves Figure 4–18
plt.figure(figsize=(9, 3.5))
plt.subplot(121)
plot_model(Lasso, polynomial=False, alphas=(0, 0.1, 1), random_state=42)
plt.ylabel("$y$  ", rotation=0)
plt.subplot(122)
plot_model(Lasso, polynomial=True, alphas=(0, 1e-2, 1), random_state=42)
plt.gca().axes.yaxis.set_ticklabels([])
plt.show()

# 4. Logistic Regression

 - Logistic Regression (also called Logit Regression) is commonly used to estimate the probability that an instance belongs to a particular class (e.g., what is the probability that this email is spam?). 
 - If the estimated probability is greater than 50%, then the model predicts that the instance belongs to that class (called the positive class, labeled “1”), and otherwise it predicts that it does not (i.e., it belongs to the negative class, labeled “0”). 
 - This makes it a binary _classifier_ (although it is a _regression_ algorithm).

 ## Estimating probabilities

 Just like a Linear Regression model, a Logistic Regression model computes a weighted sum of the input features (plus a bias term), but instead of outputting the result directly like the Linear Regression model does, it outputs the logistic of this result.

 $$
 \hat{p} = h_{\theta}(x) = \sigma(\mathbf{x^T}\mathbf{\theta})
 $$

 where, as we have seen before, 

 $$
\mathbf{x^T}\mathbf{\theta} = θ_0x_0 + θ_1x_1 + θ_2x_2 +⋯+ θ_nx_n 
 $$
 with $x_0 = 1$

 The logistic—noted $σ(·)$— is a sigmoid function (i.e., S-shaped) that outputs a number between 0 and 1. It is defined as:

$$
\sigma(t) = \frac{1}{1 + e^{-t}}
$$

<img src="img/logistic_function_plot.png" width="50%">


Once the Logistic Regression model has estimated the probability $\hat{p} = h_{\theta}(x)$ that an instance $\mathbf{x}$ belongs to the positive class, it can make its prediction $ŷ$
easily:

$\hat{y} = 0$  if  $\hat{p} < 0.5$   

$\hat{y} = 1$  if  $\hat{p} \ge 0.5$


The way a logistic regression model is trained is outside the scope of this course but again it comes down to searching the minimum of a cost function. 

## Decision Boundaries
Let’s use the iris dataset to illustrate Logistic Regression. This is a famous dataset that contains the sepal and petal length and width of 150 iris flowers
of three different species: Iris setosa, Iris versicolor, and Iris virginica.  
 
<!-- ![](img/iris.png) -->
<img src="img/iris.png" width="50%">


Let’s try to build a classifier to detect the Iris virginica type based only on the petal width feature. First let’s load the data:

In [38]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)  # as_frame=True returns a pandas DataFrame instead of a NumPy array
# just like the MNIST dataset, the iris dataset is so popular that Scikit-Learn provides a helper function to load it. 
# This function returns a Bunch object, similar to a dictionary, containing the data and the target values. 
list(iris)

In [39]:
print(iris.DESCR)

In [40]:
iris.data.head(3)

In [41]:
iris.target.head(3)  # note that the instances are not shuffled

In [42]:
iris.target_names

Now let’s train a Logistic Regression model to predict whether or not it is a virginica (binary classifier) based on the petal width. 

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = iris.data[["petal width (cm)"]].values
y = iris.target_names[iris.target] == 'virginica'
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

Let’s look at the model’s **estimated probabilities** for flowers with **petal widths varying from 0 cm to 3 cm**.

In [46]:
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)  # generate random petal widths and reshape to get a column vector
print(X_new[0][0])

y_proba = log_reg.predict_proba(X_new)
print(y_proba) # y_proba is a matrix showing probabilities 
decision_boundary = X_new[y_proba[:, 1] >= 0.5][0, 0]

print(decision_boundary)

plt.plot(X_new, y_proba[:, 1], "g-", linewidth=2, label="Iris virginica")
plt.plot(X_new, y_proba[:, 0], "b--", linewidth=2, label="Not Iris virginica")

plt.xlabel("Petal width (cm)", fontsize=14)
plt.legend(loc="center left", fontsize=12)

In [47]:
y_proba[:, 1] >= 0.5

In [48]:
X_new[:,0]

In [49]:
print(decision_boundary)

In [50]:
y_predict = log_reg.predict(X_new)
y_predict

The figure in the book is actually a bit fancier:  

<img src="img/logistic_regression_plot.png" width="50%"/>

Observations:
- The petal width of Iris virginica flowers (represented by triangles) ranges from 1.4 cm to 2.5 cm
- The other iris flowers (represented by squares) generally have a smaller petal width, ranging from 0.1 cm to 1.8 cm. 
- There is a bit of overlap. 
- Above about 2 cm the classifier is highly confident that the flower is an Iris virginica (it outputs a high probability for that class). 
- Below 1 cm it is highly confident that it is not an Iris virginica (high probability for the “Not Iris virginica” class). 
- In between these extremes, the classifier is unsure. 
- However, if you ask it to predict the class (using the predict() method rather than the predict_proba() method), it will return whichever class is the most likely.  
- Therefore, there is a decision boundary at around 1.6 cm where both probabilities are equal to 50%: if the petal width is higher than 1.6 cm, the classifier will predict that the flower is an Iris virginica, and otherwise it will predict that it is not (even if it is not very confident):

In [53]:
log_reg.predict([[1.7], [1.5]])

Now suppose the case of more than one input feature:
- The figure below shows the same dataset, but this time displaying two features: petal width and length. 
- Once trained, the Logistic Regression classifier can, based on these two features, estimate the probability that a new flower is an Iris virginica. 
- The dashed line represents the points where the model estimates a 50% probability: this is the model’s decision boundary: it is a linear boundary. 
- Each parallel line represents the points where the model outputs a specific probability, from 15% (bottom left) to 90% (top right). 
- All the flowers beyond the top-right line have an over 90% chance of being Iris virginica, according to the model.

<img src="img/logistic_regression_contour_plot.png" width="60%">

## Softmax Regression

- The Logistic Regression model can be generalized to **support multiple classes directly**, without having to train and combine multiple binary
classifiers (as discussed in Chapter 3). 
- This is called _Softmax Regression_, or _Multinomial Logistic Regression_.

The idea is simple: when given an instance $\mathbf{x}$, the Softmax Regression model first computes a score $s_k(x)$ for each class $k$, then estimates the probability of
each class by applying the _softmax function_.

The equation to compute $s_k(x)$ for class $k$ should look familiar, as it is just like the equation for Linear Regression prediction:

$$
s_k(x) = \mathbf{x^T}\mathbf{\theta}^{(k)}
$$

where $\mathbf{\theta}^{(k)}$ is the dedicated parameter vector for class $k$. 

- Once you have computed the score of every class for the instance $\mathbf{x}$, you can estimate the probability $\hat{p}$ that the instance belongs to class $k$ by running the scores through the softmax function. 
- This function computes the  exponential of every score, then normalizes them (dividing by the sum of all the exponentials to ensure the probabilities for all class sum up to one):
$$
\hat{p}_k = \mathbf{\sigma}(\mathbf{s(x)})_k = \frac{exp(s_k(x))}{\sum_{j=1}^{K}exp(s_j(\mathbf{x}))}
$$
with:
- $K$ the number of classes
- $\mathbf{s(x)}$ a vector containing the scores of each class for the instance $\mathbf{x}$.
- $\mathbf{\sigma}(\mathbf{s(x)}))_k$ the estimated probability that the instance $x$ belongs to class $k$, given the scores of each class for that instance.


Just like the Logistic Regression classifier, the Softmax Regression classifier predicts the class with the highest estimated probability (which is
simply the class with the highest score).

Again the training algorithm itself is outside the scope of this course.

Let’s use Softmax Regression to classify the iris flowers into all three classes. 
- Scikit-Learn’s `LogisticRegression` uses softmax regression automatically when you train it on more than two classes. 
- Logistic Regression uses regularization by default. The parameter `C` (default 1.0) is the inverse of the regularization strenght and must be a positive float. Smaller values specify stronger regularization. 
- You could use grid search with cross validation to find the optimal value, but here we choose `C=30` (weak regularization). 

In [55]:
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = iris["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

softmax_reg = LogisticRegression(C=30, random_state=42)
softmax_reg.fit(X_train, y_train)

So the next time you find an iris with petals that are 5 cm long and 2 cm wide, you can ask your model to tell you what type of iris it is, and it will
answer Iris virginica (class 2) with 95.9% probability (or Iris versicolor with 4.1% probability):

In [56]:
softmax_reg.predict([[5,2]])

In [57]:
softmax_reg.predict_proba([[5,2]])

The figure below shows the resulting decision boundaries, represented by the background colors. 
  
Observations: 
- the decision boundaries between any two classes are linear. 
- the probabilities for the Iris versicolor class are represented by the curved lines (e.g., the line labeled with 0.450 represents the 45% probability boundary).
- the model can predict a class that has an estimated probability below 50%. For example, at the point where all decision boundaries meet, all classes have  an equal estimated probability of 33%.  

<img src="img/softmax_regression_contour_plot.png" width="75%">